In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tarfile
tar_file_path = '/content/drive/MyDrive/musicrecom/sorted_spec.tar'
extract_to = '/content/sorted_spec'
with tarfile.open(tar_file_path, 'r') as tar:
  tar.extractall(extract_to)

Szukanie odpowiedniej architektury

In [3]:
from ParameterizedArchitecureCNN import ParameterizedArchitectureCNN
import params
from CustomImageDataset import CustomImageDataset
from utils import train_model, test_model, save_metrics



In [4]:
import importlib
importlib.reload(params)

import sys
sys.modules.pop('ParameterizedArchitecureCNN')
from ParameterizedArchitecureCNN import ParameterizedArchitectureCNN



In [4]:
import torch.optim as optim
import torch.nn as nn
import torch


criterion = nn.BCEWithLogitsLoss()
torch.cuda.set_device(0)
device = torch.device("cuda")


In [5]:
import gc

for p in params.architecture_params:
  net = ParameterizedArchitectureCNN(p["kernel_sizes"], p["stride"], p["padding"], p["pool_size"], p["fc_sizes"], p["activation_func"], p["conv_dropout"], p["fc_dropout"]).to(device)
  optimizer = optim.Adam(net.parameters(), lr=p["learning_rate"])
  trained_model = train_model(net, device, optimizer, criterion, params.tags_list, p["epochs"])
  tp, tn, fp, fn, pred = test_model(trained_model, device, params.tags_list)
  save_metrics(p, tp, tn, fp, fn, "/content/drive/MyDrive/musicrecom/training_metrics/")
  net.cpu()
  del net
  gc.collect()
  torch.cuda.empty_cache()


In [18]:
net

ParameterizedArchitectureCNN(
  (conv_layers): ModuleList(
    (0): Conv2d(4, 8, kernel_size=(10, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(8, 16, kernel_size=(3, 10), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  )
  (pool_layers): ModuleList(
    (0-2): 3 x MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=142912, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=21, bias=True)
  )
  (activation_func): ReLU()
  (conv_dropout): Dropout2d(p=0.3, inplace=False)
  (fc_dropout): Dropout(p=0.0, inplace=False)
)

In [11]:
del net
gc.collect()
torch.cuda.empty_cache()

In [15]:
import gc

net.cpu()
del net
gc.collect()
torch.cuda.empty_cache()

In [ ]:
save_metrics(p, 1, 2, 3, 4, "./")